In [13]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
from more_itertools import flatten

from pickle import UnpicklingError

from aspects.utilities import settings
# from aspects.pipelines.aspect_analysis import AspectAnalysis

c:\users\lukau\anaconda3\envs\aspects\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
%config Completer.use_jedi = False

c:\users\lukau\anaconda3\envs\aspects\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
pd.set_option('display.max_colwidth', -1)

In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { max-width:1800px !important;width:auto; }</style>"))

In [7]:
datasets = [
    settings.AMAZON_REVIEWS_APPS_FOR_ANDROID_DATASET_JSON,
    settings.AMAZON_REVIEWS_AUTOMOTIVE_DATASET_JSON,
    settings.AMAZON_REVIEWS_AMAZON_INSTANT_VIDEO_DATASET_JSON,
    settings.AMAZON_REVIEWS_CELL_PHONES_AND_ACCESSORIES_DATASET_JSON
]

In [9]:
from pathlib import Path
# p = Path('/datasets/sentiment/aspects/sentiment-backend/aspects/results')
p = Path('../../results/')

c:\users\lukau\anaconda3\envs\aspects\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
statistics = []

for discourse_trees_df_path in tqdm(list(p.glob('review*103*/discourse_trees_df.pkl'))):
    print(discourse_trees_df_path)
    try:
        df = pd.read_pickle(discourse_trees_df_path)
    except Exception as e:
        print(f'Error with {discourse_trees_df_path}. {str(e)}')
    
    if 'aspects' in df.columns:
    
        aspect_counter = Counter(
            flatten(
                flatten(
                    (
                        df[
                            df.aspects.apply(
                                lambda aspects: list(flatten(aspects))
                            )
#                             .apply(lambda aspects: len(set(aspects)) > 1)
                        ]["aspects"]
                    )
                )
            )
        )
        aspect_counter_df = pd.DataFrame(aspect_counter.most_common(), columns=["aspect", "aspect_occurrences"])

        statistics.append({
            "Dataset Name": discourse_trees_df_path.parent.stem,
            "# of reviews": df.shape[0],
            "# of DTs properly parsed": df[df.edus.apply(lambda l: len(l) > 0)].shape[0],
            "Average length of reviews (# of words)": df.text.apply(lambda review: len(review.split())).mean(),
            "Average number of EDUs in reviews": df.edus.apply(lambda edus: len(edus)).mean(),
            "# of unique aspects": aspect_counter_df.shape[0],
            "# of aspects freq >= 10": aspect_counter_df[aspect_counter_df.aspect_occurrences >= 10].shape[0],
            "# of DTs with at least one pair of aspects": df[df.aspects.apply(
                                                                lambda aspects: list(flatten(aspects))
                                                            ).apply(lambda aspects: len(set(aspects)) > 1)
                                                        ]["aspects"].shape[0],
            "# of DTs with a single aspect": df[df.aspects.apply(
                                                    lambda aspects: list(flatten(aspects))
                                                ).apply(lambda aspects: len(set(aspects)) == 1)
                                            ]["aspects"].shape[0],
            "Average number of relations from each DT": df.rules.apply(lambda rules: len(rules)).mean(),
            "Minimum depth of DT": df.discourse_tree.apply(lambda dt: dt.height()).min(),
            "Average depth of DT": df.discourse_tree.apply(lambda dt: dt.height()).mean(),
            "Maximum depth of DT": df.discourse_tree.apply(lambda dt: dt.height()).max()
            })
    else:
        print(f"Not fully processed dataset: {discourse_trees_df_path}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 45.45it/s]

..\..\results\reviews_Apps_for_Android-103-docs\discourse_trees_df.pkl
Not fully processed dataset: ..\..\results\reviews_Apps_for_Android-103-docs\discourse_trees_df.pkl


In [19]:
pd.DataFrame(statistics)

""


In [91]:
df.discourse_tree.apply(lambda dt: dt.height())

0        2
1        4
2        6
3        7
4        5
        ..
24994    4
24995    7
24996    4
24998    8
24999    5
Name: discourse_tree, Length: 23387, dtype: int64

In [90]:
tree.height()

2

In [80]:
df.drop(columns=['concepts'], inplace=True)

In [83]:
df.rules.apply(lambda rules: len(rules)).mean()

11.74207893274041

In [74]:
df[df.aspects.apply(
        lambda aspects: list(flatten(aspects))
    ).apply(lambda aspects: len(set(aspects)) > 1)
]["aspects"].shape

(7490,)

In [77]:
df[df.aspects.apply(
        lambda aspects: list(flatten(aspects))
    ).apply(lambda aspects: len(set(aspects)) == 1)
]["aspects"].shape

(7204,)

In [66]:
df.rules.apply(lambda r: len(r)).mean()

11.74207893274041

In [63]:
df.aspects.apply(lambda aspects: len([aspect for aspect in aspects if aspect])).describe()

count    23387.000000
mean         1.193868
std          1.461133
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max         20.000000
Name: aspects, dtype: float64

In [50]:
df.text.apply(lambda review: len(review.split())).describe()

count    23387.000000
mean        58.080472
std         52.459737
min          3.000000
25%         26.000000
50%         40.000000
75%         70.000000
max        706.000000
Name: text, dtype: float64

In [40]:
df.edus.apply(lambda edus: len(edus)).describe()

count    23387.000000
mean         8.425022
std          7.438310
min          1.000000
25%          4.000000
50%          6.000000
75%         10.000000
max        102.000000
Name: edus, dtype: float64

In [45]:
aspect_counter = Counter(
    flatten(
        flatten(
            (
                df[
                    df.aspects.apply(
                        lambda aspects: list(flatten(aspects))
                    ).apply(lambda aspects: len(set(aspects)) > 1)
                ]["aspects"]
            )
        )
    )
)
aspect_counter_df = pd.DataFrame(aspect_counter.most_common(), columns=["aspect", "aspect_occurrences"])

In [49]:
aspect_counter_df.shape[0]

7144

In [51]:
aspect_counter_df[aspect_counter_df.aspect_occurrences >= 10].shape[0]

242

In [41]:
df

,text,discourse_tree,discourse_tree_ids_only,edus,sentiment,aspects,concepts,rules
0,I loved the look and the great improvement at ...,[I loved the look and the great improvement at...,"[0, 1]",[I loved the look and the great improvement at...,"[0.3333333333333333, 0.0]","[[], []]","[{'sentic': {}, 'conceptnet_io': {}}, {'sentic...",[]
1,"Put these on my 2011 can am outlander 800xt, e...","[Put these on my 2011 can am outlander 800xt ,...","[0, [1, [2, 3]]]","[Put these on my 2011 can am outlander 800xt ,...","[0.43333333333333335, 0.2857142857142857, 0.0,...","[[install], [], [power switch], []]","[{'sentic': {'install': {'install': ['set-up',...","[(2, 1, Elaboration, 0.5), (3, 1, Elaboration,..."
2,"Don't buy this item , its not a 4 window roll...","[[Do n't buy this item , its not a 4 window ro...","[[0, 1], [[(Elaboration[N][S] 2 3), 4], [5, (T...","[Do n't buy this item , its not a 4 window rol...","[0.0, -0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[[], [], [], [], [], [shipping], [], []]","[{'sentic': {}, 'conceptnet_io': {}}, {'sentic...","[(0, 1, Attribution, 0.77), (1, 0, Attribution..."
3,"Nice. A decent model, and at a better price on...","[[Nice ., [A decent model , and at a better pr...","[[0, [1, (Attribution[S][N] 2 (Elaboration[N][...","[Nice ., A decent model , and at a better pric...","[0.6, 0.3333333333333333, 0.0, 0.1363636363636...","[[], [price, amazon], [], [], [], [], [price],...","[{'sentic': {}, 'conceptnet_io': {}}, {'sentic...","[(1, 0, Explanation, 0.52), (2, 0, Explanation..."
4,I used this rating because I liked the key fob...,[[[(Explanation[N][S]\n I used this rating\n ...,"[[[(Explanation[N][S] 0 1), 2], [3, (Joint[N][...","[I used this rating, because I liked the key f...","[0.0, 0.2866666666666667, 0.0, 0.0, -0.5, 0.0,...","[[], [key fob], [], [], [], [], [], [], []]","[{'sentic': {}, 'conceptnet_io': {}}, {'sentic...","[(1, 0, Explanation, 0.74), (0, 1, Explanation..."
...,...,...,...,...,...,...,...,...
24994,Love the sound of Dixie! But it only lasted a...,"[[Love the sound of Dixie !, [But it only last...","[[0, [1, 2]], [3, 4]]","[Love the sound of Dixie !, But it only lasted...","[0.5, 0.0, 0.0, 0.0, -0.025]","[[], [], [], [], []]","[{'sentic': {}, 'conceptnet_io': {}}, {'sentic...","[(1, 0, Contrast, 0.53), (2, 0, Contrast, 0.43..."
24995,This was very easy to install. They gave me f...,"[[This was very easy to install ., [(Contrast[...","[[0, [(Contrast[S][N] 1 2), (Attribution[S][N]...","[This was very easy to install ., They gave me...","[0.5633333333333334, 0.0, -0.125, 0.0, 0.0, -0...","[[install], [], [], [], [], [], [], [], [], []...","[{'sentic': {'install': {'install': ['set-up',...","[(1, 0, Elaboration, 0.68), (2, 0, Elaboration..."
24996,This does not work well. It sounds really bad ...,"[[This does not work well ., [It sounds really...","[[0, [1, 2]], [[3, 4], [5]]]","[This does not work well ., It sounds really b...","[0.0, -0.6999999999999998, -0.1, 0.0, 0.0, 0.7...","[[], [], [], [], [dixieland song], []]","[{'sentic': {}, 'conceptnet_io': {}}, {'sentic...","[(1, 0, Elaboration, 0.54), (2, 0, Elaboration..."
24998,The instructions for install are clear and eas...,[[[The instructions for install are clear and ...,"[[[0, 1], [2, 3]], [[(Attribution[S][N]\n 4\n...",[The instructions for install are clear and ea...,"[0.26666666666666666, 0.0, 0.5, 0.0, 0.25, 0.0...","[[], [], [], [], [], [], [], [], [], [], [], [...","[{'sentic': {}, 'conceptnet_io': {}}, {'sentic...","[(1, 0, Elaboration, 0.85), (0, 1, Elaboration..."


In [16]:
import json 


for d in datasets:
    dataset = json\.load(d.open('r'))
    print(d, len(dataset))

/datasets/sentiment/aspects/sentiment-backend/aspects/data/reviews/amazon/reviews_Apps_for_Android.json 100001
/datasets/sentiment/aspects/sentiment-backend/aspects/data/reviews/amazon/reviews_Automotive.json 1373794
/datasets/sentiment/aspects/sentiment-backend/aspects/data/reviews/amazon/reviews_Amazon_Instant_Video.json 100001
/datasets/sentiment/aspects/sentiment-backend/aspects/data/reviews/amazon/reviews_Cell_Phones_and_Accessories.json 100001


In [12]:
len(dataset)

100001